<a href="https://colab.research.google.com/github/Edriczz/Edrico_data_analytic/blob/main/TitanicPred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

titanic_path = kagglehub.competition_download('titanic')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [ ]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#data loading and check missing values
train_path="/kaggle/input/titanic/train.csv"
test_path="/kaggle/input/titanic/test.csv"
# Muat dataset Anda (pastikan file 'train.csv' ada di folder yang sama)
df_train = pd.read_csv(train_path)
# Cek jumlah nilai kosong di setiap kolom
missing_values = df_train.isnull().sum()
# Urutkan hasilnya agar yang paling banyak kosong ada di atas
missing_values_sorted = missing_values[missing_values > 0].sort_values(ascending=False)
# Tampilkan hasilnya
print("Jumlah Nilai Kosong di Setiap Kolom:")
print(missing_values_sorted)

Jumlah Nilai Kosong di Setiap Kolom:
Cabin       687
Age         177
Embarked      2
dtype: int64


In [ ]:
#Data Cleaning (fill the missing values with median)
import pandas as pd

# (Salin lagi fungsi clean_data dari jawaban sebelumnya di sini)
def clean_data(df):
    df_cleaned = df.copy()
    age_median = df_cleaned['Age'].median()
    df_cleaned['Age'].fillna(age_median, inplace=True)
    embarked_mode = df_cleaned['Embarked'].mode()[0]
    df_cleaned['Embarked'].fillna(embarked_mode, inplace=True)
    fare_median = df_cleaned['Fare'].median()
    df_cleaned['Fare'].fillna(fare_median, inplace=True)
    return df_cleaned

# 1. Muat data asli
df_train = pd.read_csv(train_path)

# 2. Jalankan fungsi dan SIMPAN HASILNYA ke variabel baru
df_yang_sudah_bersih = clean_data(df_train)

# 3. Sekarang, cek variabel BARU tersebut
print("Mengecek DataFrame yang sudah dibersihkan:")
print(df_yang_sudah_bersih.isnull().sum())

# --- CONTOH PENGGUNAAN ---

# Muat dataset
df_train = pd.read_csv(train_path)

# Tampilkan jumlah nilai kosong SEBELUM cleaning
print("--- SEBELUM DATA CLEANING ---")
print(df_train.isnull().sum())
print("-" * 30)


# Jalankan fungsi data cleaning
df_cleaned = clean_data(df_train)


# Tampilkan jumlah nilai kosong SETELAH cleaning
print("\n--- SETELAH DATA CLEANING ---")
print("Jumlah nilai kosong yang tersisa:")
print(df_cleaned.isnull().sum())

Mengecek DataFrame yang sudah dibersihkan:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64
--- SEBELUM DATA CLEANING ---
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
------------------------------

--- SETELAH DATA CLEANING ---
Jumlah nilai kosong yang tersisa:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64


/tmp/ipykernel_36/1934806269.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned['Age'].fillna(age_median, inplace=True)
/tmp/ipykernel_36/1934806269.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

In [ ]:
def feature_engineering(df):
    """
    Fungsi lengkap untuk feature engineering dataset Titanic.
    Termasuk membuat Title, FamilySize, Deck, AgeGroup, dan konversi ke numerik.
    """
    # Salin dataframe agar tidak mengubah data asli
    df_processed = df.copy()

    # 1. Ekstrak 'Title' dari 'Name'
    df_processed['Title'] = df_processed['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    df_processed['Title'] = df_processed['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don',
                                                           'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    df_processed['Title'] = df_processed['Title'].replace({'Mlle':'Miss', 'Ms':'Miss', 'Mme':'Mrs'})

    # 2. Isi 'Age' berdasarkan median 'Title' (diperlukan sebelum membuat AgeGroup)
    age_map = df_processed.groupby('Title')['Age'].median()
    df_processed['Age'] = df_processed['Age'].fillna(df_processed['Title'].map(age_map))

    # 3. Buat 'FamilySize' dan 'IsAlone'
    df_processed['FamilySize'] = df_processed['SibSp'] + df_processed['Parch'] + 1
    df_processed['IsAlone'] = (df_processed['FamilySize'] == 1).astype(int)

    # 4. Buat 'Deck' dari 'Cabin'
    df_processed['Deck'] = df_processed['Cabin'].str[0].fillna('U')

    # 5. Buat 'AgeGroup'
    df_processed['AgeGroup'] = pd.cut(df_processed['Age'],
                                      bins=[0, 12, 18, 60, 100],
                                      labels=['Child', 'Teen', 'Adult', 'Senior'])

    # 6. Isi 'Embarked' & 'Fare' yang mungkin kosong (terutama di data tes)
    df_processed['Embarked'].fillna(df_processed['Embarked'].mode()[0], inplace=True)
    df_processed['Fare'].fillna(df_processed['Fare'].median(), inplace=True)

    # 7. Konversi semua fitur kategorikal ke numerik
    df_processed['Sex'] = df_processed['Sex'].map({'female': 1, 'male': 0}).astype(int)

    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    df_processed['Title'] = df_processed['Title'].map(title_mapping).fillna(0)

    # Menggunakan get_dummies untuk sisanya
    df_processed = pd.get_dummies(df_processed, columns=['Embarked', 'AgeGroup', 'Deck'], prefix=['Emb', 'Age', 'Deck'])

    # 8. Hapus kolom yang tidak lagi diperlukan
    drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch']
    df_processed = df_processed.drop(drop_elements, axis=1)

    return df_processed

# --- CONTOH PENGGUNAAN ---

# Muat data
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# Terapkan fungsi ke data latih
processed_train = feature_engineering(train_df)

# Terapkan fungsi ke data uji
processed_test = feature_engineering(test_df)

# Pastikan kolom data uji sama dengan data latih
train_labels = processed_train['Survived'] # Simpan label
processed_train = processed_train.drop('Survived', axis=1)

# Samakan kolom
train_cols = processed_train.columns
test_cols = processed_test.columns
missing_in_test = set(train_cols) - set(test_cols)
for c in missing_in_test:
    processed_test[c] = 0
processed_test = processed_test[train_cols]

# Tampilkan hasil akhir
print("Data Latih Setelah Feature Engineering:")
print(processed_train.head())

Data Latih Setelah Feature Engineering:
   Pclass  Sex   Age     Fare  Title  FamilySize  IsAlone  Emb_C  Emb_Q  \
0       3    0  22.0   7.2500      1           2        0  False  False   
1       1    1  38.0  71.2833      3           2        0   True  False   
2       3    1  26.0   7.9250      2           1        1  False  False   
3       1    1  35.0  53.1000      3           2        0  False  False   
4       3    0  35.0   8.0500      1           1        1  False  False   

   Emb_S  ...  Age_Senior  Deck_A  Deck_B  Deck_C  Deck_D  Deck_E  Deck_F  \
0   True  ...       False   False   False   False   False   False   False   
1  False  ...       False   False   False    True   False   False   False   
2   True  ...       False   False   False   False   False   False   False   
3   True  ...       False   False   False    True   False   False   False   
4   True  ...       False   False   False   False   False   False   False   

   Deck_G  Deck_T  Deck_U  
0   False   False 

/tmp/ipykernel_36/1142771883.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed['Embarked'].fillna(df_processed['Embarked'].mode()[0], inplace=True)
/tmp/ipykernel_36/1142771883.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].met

In [ ]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# Asumsikan Anda sudah memiliki DataFrame 'processed_train' dan 'processed_test'
# dari langkah feature engineering sebelumnya.

# Pisahkan fitur (X) dan target (y) dari data latih
X_train = processed_train
y_train = train_labels

# Data uji kita hanya berisi fitur, kita sebut saja X_test
X_test = processed_test.copy()

In [ ]:
#Using random forest
from sklearn.ensemble import RandomForestClassifier

# 1. Inisialisasi model
# n_estimators=100 artinya kita akan membangun 100 pohon keputusan.
# random_state=42 agar hasilnya konsisten setiap kali dijalankan.
model = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=42)

# 2. Latih model dengan data Anda
# .fit() adalah perintah untuk "belajar dari data"
model.fit(X_train, y_train)

print("Model Random Forest berhasil dilatih!")

Model Random Forest berhasil dilatih!


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Asumsikan Anda sudah memiliki X_train dan y_train dari langkah sebelumnya

# 1. Bagi data untuk training dan validasi (ini penting untuk early stopping)
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)

# 2. Inisialisasi model XGBoost Classifier
model_xgb = xgb.XGBClassifier(
    n_estimators=1000,          # Jumlah pohon yang akan dibuat (bisa banyak)
    learning_rate=0.05,         # Seberapa cepat model belajar (nilai kecil lebih baik)
    early_stopping_rounds=10,   # Berhenti jika performa di data validasi tidak membaik setelah 10 ronde
    use_label_encoder=False,    # Opsi teknis untuk menghindari warning
    eval_metric='logloss'       # Metrik untuk evaluasi saat training
)

# 3. Latih model
# Kita memasukkan data validasi (X_val, y_val) ke dalam eval_set
# agar model bisa memonitor performanya dan berhenti pada saat yang tepat.
model_xgb.fit(
    X_train_split,
    y_train_split,
    eval_set=[(X_val, y_val)],
    verbose=False  # Set ke True jika ingin melihat proses training per ronde
)

# 4. Lakukan prediksi pada data validasi
val_predictions_xgb = model_xgb.predict(X_val)

# 5. Evaluasi performa model
accuracy_xgb = accuracy_score(y_val, val_predictions_xgb)
print(f"Akurasi model XGBoost: {accuracy_xgb:.4f}")
print("-" * 30)
print("Classification Report XGBoost:")
print(classification_report(y_val, val_predictions_xgb))

Akurasi model XGBoost: 0.8436
------------------------------
Classification Report XGBoost:
              precision    recall  f1-score   support

           0       0.83      0.91      0.87       105
           1       0.86      0.74      0.80        74

    accuracy                           0.84       179
   macro avg       0.85      0.83      0.83       179
weighted avg       0.84      0.84      0.84       179



In [ ]:
import pandas as pd
import xgboost as xgb

# Asumsikan variabel X_train, y_train, dan X_test sudah ada dan siap digunakan.

# 1. Latih ulang model dengan SELURUH data latih untuk performa maksimal.
#    Kita menggunakan parameter yang sama atau yang sudah di-tuning.
print("Melatih ulang model dengan seluruh data latih...")
final_model_xgb = xgb.XGBClassifier(
    n_estimators=150,           # Jumlah pohon bisa disesuaikan berdasarkan hasil validasi
    learning_rate=0.05,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

# Melatih model dengan X_train dan y_train secara penuh
final_model_xgb.fit(X_train, y_train)
print("Model final berhasil dilatih.")


# 2. Buat prediksi pada data uji (X_test)
final_predictions = final_model_xgb.predict(X_test)


# 3. Siapkan DataFrame untuk submission
#    Kita perlu memuat 'test.csv' yang asli untuk mendapatkan kolom 'PassengerId'
original_test_df = pd.read_csv(test_path)

submission_df = pd.DataFrame({
    'PassengerId': original_test_df['PassengerId'],
    'Survived': final_predictions
})


# 4. Simpan ke file CSV
#    Parameter index=False sangat penting agar tidak ada kolom indeks tambahan
submission_df.to_csv('submission_xgboost.csv', index=False)

print("\nFile 'submission_xgboost.csv' berhasil dibuat dan siap diunggah ke Kaggle!")

Melatih ulang model dengan seluruh data latih...
Model final berhasil dilatih.

File 'submission_xgboost.csv' berhasil dibuat dan siap diunggah ke Kaggle!


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Bagi data latih menjadi 80% untuk training dan 20% untuk validasi
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)

# Latih model HANYA pada data split
model.fit(X_train_split, y_train_split)

# Lakukan prediksi pada data validasi (yang tidak dilihat model saat latihan)
val_predictions = model.predict(X_val)

# Cek akurasinya
accuracy = accuracy_score(y_val, val_predictions)
print(f"Akurasi model pada data validasi: {accuracy:.4f}")

Akurasi model pada data validasi: 0.8324


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Asumsikan X_train, y_train, dan model sudah ada dari langkah sebelumnya

# Bagi data latih menjadi 80% untuk training dan 20% untuk validasi
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)

# Latih model HANYA pada data split
model.fit(X_train_split, y_train_split)

# Lakukan prediksi pada data validasi
val_predictions = model.predict(X_val)

# --- METRIK EVALUASI ---

# 1. Cek Akurasi (seperti sebelumnya)
accuracy = accuracy_score(y_val, val_predictions)
print(f"Akurasi Model: {accuracy:.4f}")
print("-" * 30)

# 2. Tampilkan Confusion Matrix
print("Confusion Matrix:")
cm = confusion_matrix(y_val, val_predictions)
print(cm)
print("\nPenjelasan Confusion Matrix:")
print(f"True Negative (TN): {cm[0][0]}  | False Positive (FP): {cm[0][1]}")
print(f"False Negative (FN): {cm[1][0]} | True Positive (TP): {cm[1][1]}")
print("\nArtinya:")
print(f"- Model dengan benar menebak {cm[0][0]} orang Tidak Selamat.")
print(f"- Model dengan benar menebak {cm[1][1]} orang Selamat.")
print(f"- Model salah menebak {cm[1][0]} orang sebagai Tidak Selamat (padahal Selamat).")
print(f"- Model salah menebak {cm[0][1]} orang sebagai Selamat (padahal Tidak Selamat).")
print("-" * 30)


# 3. Tampilkan Classification Report (Precision, Recall, F1-Score)
print("Classification Report:")
print(classification_report(y_val, val_predictions))

Akurasi Model: 0.8324
------------------------------
Confusion Matrix:
[[91 14]
 [16 58]]

Penjelasan Confusion Matrix:
True Negative (TN): 91  | False Positive (FP): 14
False Negative (FN): 16 | True Positive (TP): 58

Artinya:
- Model dengan benar menebak 91 orang Tidak Selamat.
- Model dengan benar menebak 58 orang Selamat.
- Model salah menebak 16 orang sebagai Tidak Selamat (padahal Selamat).
- Model salah menebak 14 orang sebagai Selamat (padahal Tidak Selamat).
------------------------------
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.87      0.86       105
           1       0.81      0.78      0.79        74

    accuracy                           0.83       179
   macro avg       0.83      0.83      0.83       179
weighted avg       0.83      0.83      0.83       179



In [ ]:
# 3. Lakukan prediksi pada data uji
predictions = model.predict(X_test)

# 4. Buat file submission sesuai format Kaggle
# Kita butuh PassengerId dari file test.csv asli
test_df = pd.read_csv(test_path)

submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": predictions
})

# Simpan ke file .csv
submission.to_csv('submission.csv', index=False)

print("File submission.csv berhasil dibuat!")

File submission.csv berhasil dibuat!
